## Now Training Multiple Models and Comparing Performance Using PyCaret
- GitHub link: https://github.com/pycaret/pycaret
- Documentation and tutorials also found on the GitHub

In [1]:
# General imports
import pycaret
from pycaret.classification import *
import pandas as pd
pycaret.__version__

'3.3.2'

In [2]:
# load in data 
train_data = pd.read_csv("../motum/data_for_pycaret_train.csv")
test_data = pd.read_csv("../motum/data_for_pycaret_test.csv")
print(train_data.shape)
print(test_data.shape)

# initializing pycaret setup
s = setup(
    data=train_data,
    test_data=test_data,            
    target="label",
    session_id=123,
    fold_strategy="groupkfold",     # we are doing a group-aware cross validation
    fold_groups="subject_id",     
    ignore_features=["subject_id"], # keep for grouping, not modeling
    fold=8, # number of CV folds
    index=False
)

(153, 19)
(39, 19)


,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Target mapping,"glioma: 0, metastasis: 1"
4,Original data shape,"(192, 19)"
5,Transformed data shape,"(192, 18)"
6,Transformed train set shape,"(153, 18)"
7,Transformed test set shape,"(39, 18)"
8,Ignore features,1
9,Numeric features,17


In [3]:
# initiating classification experiment class
exp = ClassificationExperiment()
type(exp)

# initiating setup on exp
exp.setup(
    data=train_data,
    test_data=test_data,            
    target="label",
    session_id=123,
    fold_strategy="groupkfold",     # we are doing a group-aware cross validation
    fold_groups="subject_id",     
    ignore_features=["subject_id"], # keep for grouping, not modeling
    fold=8, # number of CV folds
    index=False
)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Target mapping,"glioma: 0, metastasis: 1"
4,Original data shape,"(192, 19)"
5,Transformed data shape,"(192, 18)"
6,Transformed train set shape,"(153, 18)"
7,Transformed test set shape,"(39, 18)"
8,Ignore features,1
9,Numeric features,17


In [4]:
# comparing these basline models now
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7371,0.7534,0.7371,0.7781,0.7292,0.4589,0.4885,0.0188
qda,Quadratic Discriminant Analysis,0.6756,0.7239,0.6756,0.7060,0.6503,0.3308,0.3521,0.0050
lr,Logistic Regression,0.6706,0.7252,0.6706,0.7283,0.6648,0.3452,0.3826,0.4138
catboost,CatBoost Classifier,0.6448,0.7150,0.6448,0.6951,0.6418,0.2819,0.3169,0.2225
ridge,Ridge Classifier,0.6389,0.6877,0.6389,0.6780,0.6303,0.2842,0.3062,0.0075
lda,Linear Discriminant Analysis,0.6210,0.6581,0.6210,0.6607,0.6155,0.2420,0.2663,0.0050
rf,Random Forest Classifier,0.6171,0.6877,0.6171,0.6443,0.6093,0.2189,0.2360,0.0213
lightgbm,Light Gradient Boosting Machine,0.5952,0.6804,0.5952,0.6936,0.5751,0.2121,0.2709,0.0700
dt,Decision Tree Classifier,0.5655,0.6010,0.5655,0.6669,0.5207,0.1828,0.2307,0.0050
dummy,Dummy Classifier,0.5655,0.5000,0.5655,0.3260,0.4116,0.0000,0.0000,0.0088
